In [1]:
from nav2_simple_commander.robot_navigator import BasicNavigator
import rclpy

rclpy.init()
nav = BasicNavigator()

In [2]:
nav.waitUntilNav2Active()

[INFO] [1761100549.130599805] [basic_navigator]: Nav2 is ready for use!


In [3]:
import math
import tf_transformations

def get_quaternion_from_yaw(yaw_degrees):
    yaw_radians = math.radians(yaw_degrees)

    quaternion = tf_transformations.quaternion_from_euler(0, 0, yaw_radians)

    return quaternion

In [4]:
from geometry_msgs.msg import PoseStamped

initial_yaw = 0
q = get_quaternion_from_yaw(initial_yaw)

initial_pose = PoseStamped()
initial_pose.header.frame_id = "map"
initial_pose.header.stamp = nav.get_clock().now().to_msg()
initial_pose.pose.position.x = 0.0
initial_pose.pose.position.y = 0.0
initial_pose.pose.position.z = 0.0
initial_pose.pose.orientation.x = q[0]
initial_pose.pose.orientation.y = q[1]
initial_pose.pose.orientation.z = q[2]
initial_pose.pose.orientation.w = q[3]
nav.setInitialPose(initial_pose)

[INFO] [1761100691.241287196] [basic_navigator]: Publishing Initial Pose


In [5]:
sub_amcl = rclpy.create_node('sub_amcl')

In [6]:
def amcl_callback(msg) :
    print("===")
    print(msg.pose.pose.position)
    print(msg.pose.pose.orientation)

In [7]:
from geometry_msgs.msg import PoseWithCovarianceStamped

sub_amcl.create_subscription(PoseWithCovarianceStamped, 'amcl_pose', amcl_callback, 10)

In [8]:
from geometry_msgs.msg import PoseStamped

goal_yaw = 180
q = get_quaternion_from_yaw(goal_yaw)

goal_pose = PoseStamped()
goal_pose.header.frame_id = "map"
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.position.x = 0.0
goal_pose.pose.position.y = 0.0
goal_pose.pose.position.z = 0.0
goal_pose.pose.orientation.x = q[0]
goal_pose.pose.orientation.y = q[1]
goal_pose.pose.orientation.z = q[2]
goal_pose.pose.orientation.w = q[3]

In [9]:
nav.goToPose(goal_pose)

[INFO] [1761100840.573472344] [basic_navigator]: Navigating to goal: 0.0 0.0...


True

In [10]:
from geometry_msgs.msg import PoseStamped

goal_yaw = 90
q = get_quaternion_from_yaw(goal_yaw)

goal_pose = PoseStamped()
goal_pose.header.frame_id = "map"
goal_pose.header.stamp = nav.get_clock().now().to_msg()
goal_pose.pose.position.x = 0.0
goal_pose.pose.position.y = 1.0
goal_pose.pose.position.z = 0.0
goal_pose.pose.orientation.x = q[0]
goal_pose.pose.orientation.y = q[1]
goal_pose.pose.orientation.z = q[2]
goal_pose.pose.orientation.w = q[3]

In [11]:
nav.goToPose(goal_pose)

[INFO] [1761101037.515622921] [basic_navigator]: Navigating to goal: 0.0 1.0...


True

In [15]:
# 주행 중에 실행해서 피드백 받기
from rclpy.duration import Duration
from IPython.display import clear_output

while not nav.isTaskComplete() :
    clear_output(wait=True)
    rclpy.spin_once(sub_amcl, timeout_sec=1.0)

    feedback = nav.getFeedback()
    print("===")
    print("Distance remaining: " + "{:.2f}".format(
        feedback.distance_remaining) + "meters.")

    if Duration.from_msg(feedback.navigation_time) > Duration(seconds = 30.0) :
        nav.cancelTask()

In [16]:
# 주행 결과 확인
from nav2_simple_commander.robot_navigator import TaskResult

result=nav.getResult()
if result == TaskResult.SUCCEEDED:
    print("Goal succeeded!")
elif result == TaskResult.CANCELED :
    print("Goal was canceled!")
elif result == TaskResult.FAILED :
    print("Goal failed!")
else :
    print("Goal has an invalid return status!")
    

Goal succeeded!


In [ ]:
# Waypoint list 만들기

from geometry_msgs.msg import PoseStamped

goal_pose_list = []

goal_yaw1 = 0
q1 = get_quaternion_from_yaw(goal_yaw1)

goal_pose1 = PoseStamped()
goal_pose1.header.frame_id = "map"
goal_pose1.header.stamp = nav.get_clock().now().to_msg()
goal_pose1.pose.position.x = 1.31
goal_pose1.pose.position.y = 1.0
# goal_pose1.pose.position.z = 0.0
goal_pose1.pose.orientation.x = q1[0]
goal_pose1.pose.orientation.y = q1[1]
goal_pose1.pose.orientation.z = q1[2]
goal_pose1.pose.orientation.w = q1[3]
goal_pose_list.append(goal_pose1)


goal_yaw2 = 180
q2 = get_quaternion_from_yaw(goal_yaw2)

goal_pose2 = PoseStamped()
goal_pose2.header.frame_id = "map"
goal_pose2.header.stamp = nav.get_clock().now().to_msg()
goal_pose2.pose.position.x = -1.34
goal_pose2.pose.position.y = 1.0
# goal_pose2.pose.position.z = 0.0
goal_pose2.pose.orientation.x = q2[0]
goal_pose2.pose.orientation.y = q2[1]
goal_pose2.pose.orientation.z = q2[2]
goal_pose2.pose.orientation.w = q2[3]
goal_pose_list.append(goal_pose2)


goal_yaw3 = 45
q3 = get_quaternion_from_yaw(goal_yaw3)

goal_pose3 = PoseStamped()
goal_pose3.header.frame_id = "map"
goal_pose3.header.stamp = nav.get_clock().now().to_msg()
goal_pose3.pose.position.x = -1.39
goal_pose3.pose.position.y = -0.36
# goal_pose3.pose.position.z = 0.0
goal_pose3.pose.orientation.x = q3[0]
goal_pose3.pose.orientation.y = q3[1]
goal_pose3.pose.orientation.z = q3[2]
goal_pose3.pose.orientation.w = q3[3]
goal_pose_list.append(goal_pose3)

In [19]:
nav_start = nav.get_clock().now()
nav.followWaypoints(goal_pose_list)

[INFO] [1761101854.013903271] [basic_navigator]: Following 3 goals....


True

In [20]:
from rclpy.duration import Duration

while not nav.isTaskComplete() :
    clear_output(wait=True)
    rclpy.spin_once(sub_amcl, timeout_sec=1.0)

    feedback = nav.getFeedback()

    print("===")
    print(
        "Executing current waypoint: "
        + str(feedback.current_waypoint + 1)
        + "/"
        + str(len(goal_pose_list))
    )
    now = nav.get_clock().now()

    if now - nav_start > Duration(seconds = 300.0) :
        nav.cancelTask()

===
Executing current waypoint: 3/3


[INFO] [1761102154.579948992] [basic_navigator]: Canceling current task.
